# Fine-tuning Gemma for Kinyarwanda: Sex Education & Mental Health for Young Girls

This notebook fine-tunes the Gemma model for Kinyarwanda to help young girls with:
- **Sex Education**: Age-appropriate, culturally sensitive information about reproductive health
- **Self-Protection**: Guidance on how to protect themselves and behave safely in society
- **Mental Health Awareness**: Counseling and support for mental well-being

## Overview
- Load pre-created Kinyarwanda instruction-response pairs from local JSONL files
- Fine-tune Gemma model on the dataset
- Focus on empowering young girls with knowledge, safety awareness, and mental health support
- Culturally appropriate and age-appropriate content in Kinyarwanda

## Step 1: Install Required Libraries


In [ ]:
%pip install -q transformers accelerate bitsandbytes peft datasets
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

## Step 2: Load Local Counseling Dataset (JSONL)

Load the pre-created Kinyarwanda counseling dataset from JSONL files in the data directory.

In [ ]:
import json
from pathlib import Path

# Path to data directory
data_dir = Path("./data")

# Load training and validation datasets from JSONL files
train_file = data_dir / "kinyarwanda_counseling.train.jsonl"
val_file = data_dir / "kinyarwanda_counseling.validation.jsonl"

def load_jsonl(filepath):
    """Load data from JSONL file."""
    data = []
    if filepath.exists():
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():
                    data.append(json.loads(line))
    return data

# Load datasets
print("Loading Kinyarwanda counseling dataset from JSONL files...")
train_data = load_jsonl(train_file)
val_data = load_jsonl(val_file)

print(f"Loaded {len(train_data)} training pairs")
print(f"Loaded {len(val_data)} validation pairs")
print(f"Total pairs: {len(train_data) + len(val_data)}")

# Extract instructions and responses
kinyarwanda_instructions = [item["instruction"] for item in train_data + val_data]
kinyarwanda_responses = [item["output"] for item in train_data + val_data]

# Display sample generated content
print("\n=== Sample Generated Content ===")
for i in range(min(3, len(train_data))):
    record = train_data[i]
    print(f"\nTopic: {record['topic']}")
    print(f"Scenario: {record['scenario']}")
    print(f"\nInstruction: {record['instruction']}")
    print(f"\nResponse: {record['output'][:200]}...")
    print("-" * 80)

# Per-topic statistics
from collections import Counter
topic_counts = Counter(item["topic"] for item in train_data + val_data)
print("\n=== Dataset Statistics ===")
print(f"Total topics: {len(topic_counts)}")
print(f"Pairs per topic: {dict(topic_counts)}")
print(f"\nDomain distribution:")
domain_counts = Counter(item["domain"] for item in train_data + val_data)
print(dict(domain_counts))


## Step 3: Verify Dataset

The dataset is saved in JSONL format in the `data/` directory:
- `data/kinyarwanda_counseling.train.jsonl` (training set)
- `data/kinyarwanda_counseling.validation.jsonl` (validation set)

In [ ]:
print("Dataset is saved in JSONL format in the data/ directory.")
print("Training data: data/kinyarwanda_counseling.train.jsonl")
print("Validation data: data/kinyarwanda_counseling.validation.jsonl")
print(f"Total pairs loaded: {len(kinyarwanda_instructions)}")

## Step 4: Format Data for Gemma Model

Gemma uses a specific chat template format with `<start_of_turn>` and `<end_of_turn>` tokens.

In [ ]:
def format_gemma_instruction(instruction, response):
    """
    Format instruction and response in Gemma chat template format.
    
    Args:
        instruction: User instruction
        response: Model response
    
    Returns:
        Formatted string for training
    """
    formatted = f"<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n{response}<end_of_turn>"
    return formatted

# Create formatted training examples
training_examples = []
for inst, resp in zip(kinyarwanda_instructions, kinyarwanda_responses):
    formatted = format_gemma_instruction(inst, resp)
    training_examples.append(formatted)

print(f"Created {len(training_examples)} training examples")
print("\n=== Sample Formatted Example ===")
print(training_examples[0])


## Step 5: Create Hugging Face Dataset

In [ ]:
from datasets import Dataset

# Create a dataset from the training examples
dataset_dict = {
    "text": training_examples
}

dataset = Dataset.from_dict(dataset_dict)

print(f"Dataset created with {len(dataset)} examples")
print(f"Sample entry:\n{dataset[0]['text']}")


## Step 6: Load Gemma-2 Model and Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
import torch

# Model configuration
model_name = "google/gemma-2-2b-it"  # Using Gemma-2 2B instruction-tuned model
# Alternative: "google/gemma-2-9b-it" for larger model (requires more memory)

login()

# Configure quantization for memory efficiency (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("Loading model with 4-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

print("Model and tokenizer loaded successfully!")
print(f"Model: {model_name}")
print(f"Model dtype: {model.dtype}")


## Step 7: Configure PEFT (Parameter-Efficient Fine-Tuning) with LoRA

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

# Prepare model for training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,  # LoRA alpha scaling
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()


## Step 8: Tokenize Dataset

In [ ]:
def tokenize_function(examples):
    """
    Tokenize the text examples for training.
    """
    # Tokenize with truncation and padding
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt",
    )
    
    # For causal LM, labels are the same as input_ids
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

# Tokenize the dataset
print("Tokenizing dataset...")
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names,
)

print(f"Tokenized dataset created with {len(tokenized_dataset)} examples")
print(f"Sample tokenized keys: {tokenized_dataset[0].keys()}")


## Step 9: Configure Training Arguments

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Training arguments
training_args = TrainingArguments(
    output_dir="./gemma-kinyarwanda-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    learning_rate=2e-4,
    fp16=False,  # Use bfloat16 instead
    bf16=True,   # Better for Gemma models
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    load_best_model_at_end=False,
    report_to="tensorboard",
    optim="paged_adamw_8bit",  # Memory-efficient optimizer
    lr_scheduler_type="cosine",
    max_grad_norm=0.3,
)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal LM, not masked LM
)

print("Training arguments configured!")


## Step 10: Initialize Trainer and Start Fine-tuning

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Start training
print("Starting fine-tuning...")
print(f"Training on {len(tokenized_dataset)} examples")
print(f"Number of epochs: {training_args.num_train_epochs}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Gradient accumulation steps: {training_args.gradient_accumulation_steps}")

trainer.train()

print("\nFine-tuning completed!")


## Step 11: Save the Fine-tuned Model

In [ ]:
# Save the fine-tuned model
output_dir = "./gemma-kinyarwanda-finetuned-final"
print(f"Saving model to {output_dir}...")

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("Model and tokenizer saved successfully!")


## Step 12: Test the Fine-tuned Model

In [ ]:
# Load the fine-tuned model for inference
from peft import PeftModel

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

# Load fine-tuned LoRA weights
fine_tuned_model = PeftModel.from_pretrained(base_model, output_dir)
fine_tuned_model.eval()

print("Fine-tuned model loaded for inference!")


## Step 13: Generate Responses in Kinyarwanda

In [ ]:
def generate_response(prompt, max_length=512, temperature=0.7):
    """
    Generate a response using the fine-tuned model.
    
    Args:
        prompt: Input prompt in Kinyarwanda
        max_length: Maximum length of generated text
        temperature: Sampling temperature
    
    Returns:
        Generated response
    """
    # Format prompt in Gemma chat format
    formatted_prompt = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    
    # Tokenize
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(fine_tuned_model.device)
    
    # Generate
    with torch.no_grad():
        outputs = fine_tuned_model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # Decode response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    # Extract only the model's response
    if "<start_of_turn>model\n" in full_response:
        response = full_response.split("<start_of_turn>model\n")[-1]
        response = response.split("<end_of_turn>")[0].strip()
    else:
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response

# Test with sample prompts from the loaded dataset
test_prompts = kinyarwanda_instructions[:3]

print("=== Testing Fine-tuned Model ===\n")
for i, prompt in enumerate(test_prompts, 1):
    print(f"Test {i}:")
    print(f"Prompt: {prompt}")
    response = generate_response(prompt)
    print(f"Response: {response}")
    print("-" * 80)
    print()

## Step 14: Compare with Base Model (Optional)

Compare responses from the base model vs fine-tuned model to see the improvement.

In [ ]:
# Load base model for comparison
base_model_for_comparison = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

def generate_base_response(prompt, max_length=512, temperature=0.7):
    """Generate response using base model."""
    formatted_prompt = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(base_model_for_comparison.device)
    
    with torch.no_grad():
        outputs = base_model_for_comparison.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    if "<start_of_turn>model\n" in full_response:
        response = full_response.split("<start_of_turn>model\n")[-1]
        response = response.split("<end_of_turn>")[0].strip()
    else:
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response

# Compare responses with a prompt from the dataset
test_prompt = kinyarwanda_instructions[0]

print("=== Model Comparison ===\n")
print(f"Prompt: {test_prompt}\n")

print("Base Model Response:")
base_response = generate_base_response(test_prompt)
print(base_response)
print("\n" + "-" * 80 + "\n")

print("Fine-tuned Model Response:")
fine_tuned_response = generate_response(test_prompt)
print(fine_tuned_response)
print("\n" + "=" * 80)

## Step 15: Save Model to Google Drive (Optional)

If running on Google Colab and want to save the model to Google Drive for later use.

In [ ]:
# Save model locally first
print(f"✅ Model saved locally at: {output_dir}")
print("\nNext: We'll merge LoRA adapters and convert to GGUF format for Ollama deployment")

## Notes and Tips

1. **Dataset**:
   - The dataset is pre-created and stored in `data/kinyarwanda_counseling.train.jsonl` and `data/kinyarwanda_counseling.validation.jsonl`
   - Content covers sex education, self-protection, and mental health topics for young girls (ages 10-16)
   - All content is in natural Kinyarwanda, culturally sensitive, and age-appropriate

2. **Memory Management**: If you run out of memory, try:
   - Reducing `per_device_train_batch_size`
   - Increasing `gradient_accumulation_steps`
   - Using a smaller model (e.g., gemma-2-2b instead of 9b)

3. **Training Time**: 
   - Fine-tuning typically takes 10-30 minutes depending on dataset size and hardware
   - GGUF conversion adds another 5-10 minutes

4. **GGUF Conversion**:
   - Q4_K_M quantization provides good balance between size and quality
   - Other options: Q2_K (smaller, lower quality), Q5_K_M (larger, higher quality), Q8_0 (largest, best quality)
   - Final GGUF file will be significantly smaller than the merged model

5. **Evaluation**: 
   - Test with real scenarios that young girls might ask about
   - Evaluate responses for cultural appropriateness, accuracy, and empathy

6. **Model Deployment**: 
   - Use with Ollama for easy local deployment
   - Can also deploy on servers or integrate with applications
   - Ensure proper disclaimers about limitations

7. **Important Disclaimers and Safety Considerations**: 
   - This model provides educational information only and is NOT a replacement for professional services
   - Always direct users to trusted adults and professionals when appropriate
   - Include disclaimers in any production deployment

In [ ]:
import os

print("🎉 " + "="*60)
print("🎉 MODEL FINE-TUNING AND CONVERSION COMPLETE!")
print("🎉 " + "="*60)

print("\n📁 Files Created:\n")

# Check and display files
files_to_check = [
    ("gemma-kinyarwanda.gguf", "GGUF model file for Ollama"),
    ("Modelfile", "Ollama configuration file"),
    ("ollama_usage_instructions.md", "Usage instructions"),
]

for filename, description in files_to_check:
    if os.path.exists(filename):
        size = os.path.getsize(filename)
        if size > 1024*1024*1024:
            size_str = f"{size / (1024**3):.2f} GB"
        elif size > 1024*1024:
            size_str = f"{size / (1024**2):.2f} MB"
        else:
            size_str = f"{size / 1024:.2f} KB"
        print(f"  ✅ {filename:<35} ({size_str}) - {description}")
    else:
        print(f"  ❌ {filename:<35} - NOT FOUND")

print("\n🚀 Next Steps:\n")
print("1. Install Ollama: https://ollama.ai")
print("2. Run: ollama create gemma-kinyarwanda -f Modelfile")
print("3. Test: ollama run gemma-kinyarwanda")
print("4. Try a query: ollama run gemma-kinyarwanda \"Amaraso yanjye yatangiye kuza, ni ibisanzwe?\"")

print("\n📖 See 'ollama_usage_instructions.md' for detailed guidance")

print("\n💡 Model Purpose:")
print("   • Provides culturally appropriate guidance in Kinyarwanda")
print("   • Covers sex education, self-protection, and mental health")
print("   • Designed for young girls (ages 10-16) in rural Rwanda")
print("   • Empowers girls with knowledge and support")

print("\n⚠️  Important Reminders:")
print("   • This model provides educational information only")
print("   • Always consult professionals for personal health concerns")
print("   • Encourage seeking help from trusted adults when needed")

print("\n✅ Your Gemma Kinyarwanda counseling model is ready for deployment!")
print("💪 Empowering young girls with knowledge in their native language.\n")

## Step 21: Final Summary

In [ ]:
# Create usage instructions
usage_content = """# Using Your Gemma Kinyarwanda Model with Ollama

## Prerequisites
- Ollama installed on your system ([https://ollama.ai](https://ollama.ai))
- GGUF file and Modelfile in the same directory

## Quick Start

### 1. Create the Ollama Model
```bash
ollama create gemma-kinyarwanda -f Modelfile
```

### 2. Run the Model
```bash
# Start interactive chat
ollama run gemma-kinyarwanda

# Or run a single query
ollama run gemma-kinyarwanda "Amaraso yanjye yatangiye kuza, ni ibisanzwe?"
```

## Example Queries in Kinyarwanda

### Sex Education
```bash
ollama run gemma-kinyarwanda "Ni iki cyo gukora igihe mfite amaraso bwa mbere?"
ollama run gemma-kinyarwanda "Umubiri wanjye urahinduka, iki ni ibisanzwe?"
```

### Mental Health
```bash
ollama run gemma-kinyarwanda "Numva ntekereza cyane, nshobora gukora iki?"
ollama run gemma-kinyarwanda "Ndi mu ngorane, nshobora kubwira nde?"
```

### Self-Protection
```bash
ollama run gemma-kinyarwanda "Umuntu arantumbagiza, nshobora gukora iki?"
ollama run gemma-kinyarwanda "Ni gute nshobora kwirinda ku murongo?"
```

## Model Information

- **Base Model**: google/gemma-2-2b-it
- **Quantization**: Q4_K_M
- **Language**: Kinyarwanda
- **Domain**: Sex education, self-protection, mental health
- **Target Audience**: Young girls (ages 10-16) in Rwanda

## Important Disclaimer

⚠️ **Medical Disclaimer**: This model provides educational information only and is NOT a replacement for:
- Professional medical or mental health services
- Professional counseling or therapy
- Emergency services

Always consult qualified professionals for personal health concerns.

## Troubleshooting

### Model Not Found
```bash
ollama list  # Check if model is created
ollama rm gemma-kinyarwanda  # Remove if needed
ollama create gemma-kinyarwanda -f Modelfile  # Recreate
```

### Performance Issues
```bash
# Reduce context window if memory is limited
ollama run gemma-kinyarwanda --num-ctx 1024
```

## Files Included

- `gemma-kinyarwanda.gguf` - The model file (required)
- `Modelfile` - Ollama configuration (required)
- `ollama_usage_instructions.md` - This file
"""

# Save usage instructions
usage_path = "./ollama_usage_instructions.md"
with open(usage_path, 'w') as f:
    f.write(usage_content)

print(f"✅ Usage instructions created: {usage_path}")

## Step 20: Create Usage Instructions

In [ ]:
# Create Modelfile for Ollama
modelfile_content = """# Modelfile for Gemma Kinyarwanda Counseling Model
# Fine-tuned for sex education, self-protection, and mental health counseling

FROM ./gemma-kinyarwanda.gguf

# Model parameters
PARAMETER temperature 0.7
PARAMETER top_p 0.95
PARAMETER top_k 40
PARAMETER num_ctx 2048
PARAMETER repeat_penalty 1.1
PARAMETER repeat_last_n 64

# System prompt for Kinyarwanda counseling
SYSTEM \"\"\"You are a helpful, culturally sensitive assistant in Kinyarwanda that provides age-appropriate guidance for young girls (ages 10-16) on:

- Sex education and reproductive health (medically accurate, age-appropriate)
- Self-protection and personal safety
- Mental health awareness and counseling
- Social behavior and healthy relationships
- Body awareness, consent, and boundaries

IMPORTANT GUIDELINES:
- Always respond in natural, fluent Kinyarwanda
- Be culturally sensitive and appropriate for Rwandan context
- Provide age-appropriate information for young girls (10-16 years)
- Be empathetic, supportive, and non-judgmental
- Give medically accurate information when discussing health topics
- Empower young girls with knowledge and self-confidence
- Encourage seeking help from trusted adults when appropriate
- Respect cultural values while providing necessary information

DISCLAIMER: This AI provides educational information only and is NOT a replacement for professional medical, psychological, or counseling services.\"\"\"

# Template for Gemma chat format
TEMPLATE \"\"\"{{{{ if .System }}}}{{{{ start_of_turn }}}}system
{{{{ .System }}}}{{{{ end_of_turn }}}}
{{{{ end }}}}{{{{ start_of_turn }}}}user
{{{{ .Prompt }}}}{{{{ end_of_turn }}}}
{{{{ start_of_turn }}}}model
{{{{ .Response }}}}\"\"\"

# Metadata
LICENSE "Apache 2.0"
DESCRIPTION "Gemma model fine-tuned for Kinyarwanda counseling on sex education, self-protection, and mental health for young girls. Provides culturally appropriate, age-appropriate guidance in Kinyarwanda language."
"""

# Save Modelfile
modelfile_path = "./Modelfile"
with open(modelfile_path, 'w') as f:
    f.write(modelfile_content)

print(f"✅ Modelfile created: {modelfile_path}")
print(f"\n📄 Modelfile content:\n")
print(modelfile_content)

## Step 19: Create Modelfile for Ollama

In [ ]:
import sys

# Configuration
GGUF_OUTPUT_PATH = "./gemma-kinyarwanda.gguf"
QUANTIZATION = "Q4_K_M"  # Good balance between size and quality

print(f"🔄 Converting model to GGUF format...")
print(f"Model path: {MERGED_MODEL_PATH}")
print(f"Output path: {GGUF_OUTPUT_PATH}")
print(f"Quantization: {QUANTIZATION}")

# Prepare conversion command
convert_script = "./llama.cpp/convert_hf_to_gguf.py"

# Check if the script exists
if not os.path.exists(convert_script):
    print(f"⚠️ Script not found at {convert_script}, trying alternative...")
    convert_script = "./llama.cpp/convert.py"

if not os.path.exists(convert_script):
    print("❌ Conversion script not found!")
    print("Available scripts in llama.cpp:")
    !ls -la llama.cpp/*.py | grep convert
else:
    # Run conversion
    cmd = [
        sys.executable, convert_script,
        MERGED_MODEL_PATH,
        "--outtype", QUANTIZATION,
        "--outfile", GGUF_OUTPUT_PATH
    ]
    
    print(f"\n🚀 Running conversion command...")
    print(f"Command: {' '.join(cmd)}\n")
    
    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print("✅ GGUF conversion completed successfully!")
        print(result.stdout)
        
        # Check file size
        if os.path.exists(GGUF_OUTPUT_PATH):
            size_mb = os.path.getsize(GGUF_OUTPUT_PATH) / (1024 * 1024)
            size_gb = size_mb / 1024
            print(f"\n📊 GGUF file size: {size_mb:.2f} MB ({size_gb:.2f} GB)")
            print(f"📁 Location: {os.path.abspath(GGUF_OUTPUT_PATH)}")
        else:
            print("⚠️ Output file was not created")
            
    except subprocess.CalledProcessError as e:
        print(f"❌ Conversion failed with exit code {e.returncode}")
        print(f"STDOUT: {e.stdout}")
        print(f"STDERR: {e.stderr}")
        print("\n💡 Tip: You may need to use a different conversion script or parameters")
    except Exception as e:
        print(f"❌ Conversion failed: {e}")

## Step 18: Convert Merged Model to GGUF Format

In [ ]:
import os
import subprocess

# Check if llama.cpp exists, if not clone it
if not os.path.exists("llama.cpp"):
    print("📥 Cloning llama.cpp repository...")
    !git clone https://github.com/ggerganov/llama.cpp.git
    print("✅ llama.cpp cloned")
else:
    print("✅ llama.cpp already exists")

# Install requirements
print("📦 Installing Python requirements for llama.cpp...")
!pip install -q -r llama.cpp/requirements.txt

# Build llama.cpp (this compiles the C++ code)
print("🔨 Building llama.cpp...")
!make -C llama.cpp

print("✅ llama.cpp setup completed!")

## Step 17: Setup llama.cpp for GGUF Conversion

In [ ]:
import time

# Configuration for conversion
MERGED_MODEL_PATH = "./gemma-kinyarwanda-merged"

print("🔀 Merging LoRA adapters with base model...")
print(f"Base model: {model_name}")
print(f"LoRA adapters: {output_dir}")
print(f"Output: {MERGED_MODEL_PATH}")

# Load base model without quantization for merging
print("\n🧠 Loading base model...")
merge_base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

# Load LoRA adapter
print("🔌 Loading LoRA adapter...")
from peft import PeftModel
merge_model = PeftModel.from_pretrained(merge_base_model, output_dir)

# Merge and unload
print("🔀 Merging LoRA weights with base model...")
merged_model = merge_model.merge_and_unload()

# Save merged model
print(f"💾 Saving merged model to: {MERGED_MODEL_PATH}")
import os
os.makedirs(MERGED_MODEL_PATH, exist_ok=True)
merged_model.save_pretrained(MERGED_MODEL_PATH)
tokenizer.save_pretrained(MERGED_MODEL_PATH)

print("✅ Model merge completed!")

# Clean up memory
del merge_base_model, merge_model, merged_model
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("🧹 Memory cleaned")

## Step 16: Merge LoRA Adapters with Base Model

---

# Part 2: Convert to GGUF Format for Ollama

Now we'll merge the LoRA adapters with the base model and convert to GGUF format for Ollama deployment.